## Mounting:

In [174]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Importing Packages

In [175]:
import sys
sys.path.append('/content/drive/MyDrive/AN')
import pandas as pd
import rafs_instance as instance
import json
import os
import os.path
from os import path
import xml.etree.ElementTree as ET
import networkx as nx
from lxml import etree
import pdb
from collections import defaultdict
import random
import numpy as np
import math
import copy
from itertools import groupby 
from random import randint
from math import exp
from math import log

# Loading data files:

In [193]:
# Specify version of order ("", "_a" or "_b")
orderVersion = "_a"

# Specify mean item amount in an order (either 1x6 or 5)
meanItemInOrder = "5"

# Specify the number of orders we receive (either 10 or 20)
orderAmount = 20

# Specify the number of items in the warehouse (either 24 or 360)
itemAmount = 360

# SPecifiy the pod amount
podAmount = 360

# Specify the policy (either "dedicated_1" or "mixed_shevels_1-5")
podPolicy = "dedicated_1"

#%% A) importing files and creating instances defined in instance_demo.py

#All files required for SKU24 and SKU360
layoutFile = r'/content/drive/MyDrive/AN/data/layout/1-1-1-2-1.xlayo'

# loading all the information about the pods
podInfoFile = '/content/drive/MyDrive/AN/data/sku' + str(itemAmount) + '/pods_infos.txt'   
# loading information about picking locations, packing stations, waypoints,
# pods 
instances = {}
instances[itemAmount,2] = r'/content/drive/MyDrive/AN/data/sku' + str(itemAmount) + '/layout_sku_' + str(itemAmount) + '_2.xml'
instanceFile = r'data/sku' + str(itemAmount) + '/layout_sku_' + str(itemAmount) + '_2.xml'

storagePolicies = {}
# loading information about item storage: contains all SKUs along with their
# attributes
storagePolicies['dedicated'] = '/content/drive/MyDrive/AN/data/sku' + str(itemAmount) + '/pods_items_' + str(podPolicy) + '.txt'
#storagePolicies['mixed'] = 'data/sku24/pods_items_mixed_shevels_1-5.txt'

# loading information about the orders: contains list of orders, with number
# of ordered items per SKU-ID
orders = {}
orders[str(orderAmount)+"_"+ str(meanItemInOrder)] =r'/content/drive/MyDrive/AN/data/sku' + str(itemAmount) + '/orders_' + str(orderAmount) + '_mean_' + str(meanItemInOrder) + '_sku_' + str(itemAmount) + orderVersion + '.xml'


# Data Processing:

In [194]:
class WarehouseDateProcessing():
    def __init__(self, warehouseInstance, batch_size = None):
        self.Warehouse = warehouseInstance # the link to the .py file
        self._InitSets(warehouseInstance, batch_size) # creates V as a dict with output station and item lists based on color and letter for each order   
        
    def preprocessingFilterPods(self, warehouseInstance):
        
        resize_pods = {}
        print("preprocessingFilterPods")
        item_id_list=[]
        for order in warehouseInstance.Orders:
            for pos in order.Positions.values():
                item = warehouseInstance.ItemDescriptions[pos.ItemDescID].Color.lower() + '/' + warehouseInstance.ItemDescriptions[pos.ItemDescID].Letter

                if item not in item_id_list:
                    item_id_list.append(item)
                

        # for dedicated
        for pod in warehouseInstance.Pods.values():
            for item in pod.Items:
                if item.ID in item_id_list:
                    resize_pods[pod.ID] = pod

        return resize_pods

    # Initialize sets and parameters           
    def _InitSets(self,warehouseInstance, batch_size):
        #V Set of nodes, including shelves V^S and stations (depots)
        # V^D (V=V^S U V^D)
        #Add output and input depots
        
        self.V__D__C = warehouseInstance.OutputStations
        self.V__D__F = {}

             
        self.V__D = {**self.V__D__C, **self.V__D__F}

        self.V__S = self.preprocessingFilterPods(warehouseInstance)
        
        #Merge dictionaries
        self.V = {**self.V__D, **self.V__S}

    def CalculateDistance(self):

        file_path = r'/content/drive/MyDrive/AN/data/distances/' + os.path.splitext(os.path.basename(self.Warehouse.InstanceFile))[0] + '.json'
        if not path.exists(file_path):
            #Create d_ij
            d_ij = {}
            for key_i, node_i in self.V.items():
                for key_j, node_j in self.V.items():  
                    
                    source = 'w'+node_i.GetPickWaypoint().ID
                    target = 'w'+node_j.GetPickWaypoint().ID
                    
                    #Calc distance with weighted shortest path
                    d_ij[(key_i,key_j)] = nx.shortest_path_length(self.Graph, source=source, target=target, weight='weight')
            
            #Parse and save
            d_ij_dict = {}
            for key,value in d_ij.items():
                i,j = key  
                if i not in d_ij_dict:
                    d_ij_dict[i]={} 
                d_ij_dict[i][j] = value
            
            with open(file_path, 'w') as fp:
                json.dump(d_ij_dict, fp)
        
        else: 
            #Load and deparse
            with open(file_path, 'r') as fp:
                d_ij_dict = json.load(fp)
            print('d_ij file %s loaded'%(file_path)) 
                
            #d_ij = tupledict()
            d_ij = {}
            for i, values in d_ij_dict.items():
                for j, dist in values.items():
                    d_ij[i,j] = dist                
                
        return d_ij
		

class Demo():
    def __init__(self, splitOrders = False):
        
        self.batch_weight = 18
        self.item_id_pod_id_dict = {}
        #[0]
        self.warehouseInstance = self.prepareData()
        self.distance_ij = self.initData()
        #[2]
        if storagePolicies.get('dedicated'):
            self.is_storage_dedicated = True
        else:
            self.is_storage_dedicated = False


	# warehouse instance
    def prepareData(self):
        print("[0] preparing all data with the standard format: ")
        #print(instances)
        #Every instance
        for key,instanceFile in instances.items():
          # the key is the previously defined tuple in instances
          # the instanceFile is the XML path
            #print(instanceFile)
            podAmount = key[0]
            depotAmount = key[1] 
            #For different orders
            for key, orderFile in orders.items():
              # the key is the previously defined tuple in orders
                orderAmount = key
                #For storage policies
                for storagePolicy, storagePolicyFile in storagePolicies.items(): # what item in what pod   
                    warehouseInstance = instance.Warehouse(layoutFile, instanceFile, podInfoFile, storagePolicyFile, orderFile)
        return warehouseInstance

	# distance
    def initData(self):
        print("[1] changing data format for the algorithm we used here: ")
        warehouse_data_processing = WarehouseDateProcessing(self.warehouseInstance)
        #Distance d_ij between two nodes i,j \in V
        d_ij = warehouse_data_processing.CalculateDistance()
        return d_ij


if __name__ == "__main__":
    _demo = Demo()





[0] preparing all data with the standard format: 
[1] changing data format for the algorithm we used here: 
preprocessingFilterPods
d_ij file /content/drive/MyDrive/AN/data/distances/layout_sku_360_2.json loaded


# Greedy


## Distances:

In [195]:
distances = _demo.distance_ij

packing_stations = list(_demo.warehouseInstance.OutputStations.keys()) # getting the packing stations as dict


## Data extraction:

In [196]:

# to extract the dict of all locations in an order
def extract_order_positions(order):
  return list(order.Positions.keys())


# to extract all the weights of the items inside an order
def extract_order_weight(order, ItemDescriptions = _demo.warehouseInstance.ItemDescriptions):
  total_weight = 0
  for pos in order.Positions.values():
    total_weight += ItemDescriptions[pos.ItemDescID].Weight * float(pos.Count)
  return total_weight

def extract_order_letter_color(order, ItemDescriptions = _demo.warehouseInstance.ItemDescriptions): 
  # color_letter = {}
  item_id_list = []
  for pos in order.Positions.values():
    item = ItemDescriptions[pos.ItemDescID].Color.lower() + '/' + ItemDescriptions[pos.ItemDescID].Letter
    # color_letter[order] = item
    if item not in item_id_list:
      item_id_list.append(item)
        # print(item_id)
  return item_id_list


In [197]:
orders_list = _demo.warehouseInstance.Orders
orders_list

In [198]:
#we need to create a nested dictionary: orders -> Item ID -> shelve ID
# 1) we can get order - > item ID(position) -> letter/color
# 2) get letter/color -> pod ID (from warehouseInstance.Pods)
# 3)combine 1 and 2 to get order -> item ID -> pod ID

#from this dictionary we can extract list of all ID for items of orders
def get_order_item_ID(orders = orders_list):
  order_item_ID = {}
  for order in orders:
    order_item_ID[order] = extract_order_positions(order)
  return order_item_ID

#from this dictionary we can extract list of all color/letter for items of orders
def get_order_item_letter_color(orders = orders_list, ItemDescriptions = _demo.warehouseInstance.ItemDescriptions):
  order_item_letter_color = {}
  for order in orders:
    order_item_letter_color[order] = extract_order_letter_color(order, ItemDescriptions)
  return order_item_letter_color



#create nested dictionary orders -> item ID -> letter/color
def get_order_itemID_letter_color(orders = orders_list):
  orders_item_ID = get_order_item_ID(orders)
  order_item_letter_color = get_order_item_letter_color(orders = orders)
  order_itemID_item_letter_color = {}
  for order in orders:
    order_itemID_item_letter_color[order] = dict(zip(orders_item_ID[order], order_item_letter_color[order]))       
  return order_itemID_item_letter_color


#create dictionary podID -> letter/color
#the dictionary will be valid for mix shelve policy too
# contains item ID and description for the items stored in each pod
def get_podID_pod_letter_color(pods_dict = _demo.warehouseInstance.Pods):
  podID_pod_letter_color = defaultdict(list)
  for podID, pod in pods_dict.items():
    for item in pod.Items:
      podID_pod_letter_color[podID] +=  [item.ID]
  return podID_pod_letter_color


#create a dictionary orders -> itemID -> podID
#This dictionary is for dedicated policy, for mix policy for each itemID will be list of podID
def get_order_itemID_podID(orders = orders_list):
  order_itemID_item_color_letter = get_order_itemID_letter_color(orders)
  podID_pod_letter_color = get_podID_pod_letter_color()
  order_itemID_podID = {}
  for order in orders:
    order_itemID_podID[order] = {}
    for itemID, letter_color in order_itemID_item_color_letter[order].items():
        for podID, list_letter_color in podID_pod_letter_color.items():
          if letter_color in list_letter_color:
            order_itemID_podID[order][itemID] = podID
  return order_itemID_podID


#for dedicated policy we need just dictionary: order -> podID:          
def get_order_podID(orders = orders_list):
  order_itemID_item_color_letter = get_order_itemID_letter_color(orders)
  podID_pod_letter_color = get_podID_pod_letter_color()
  order_podID = defaultdict(list)
  for order in orders:
    for itemID, letter_color in order_itemID_item_color_letter[order].items():
        for podID, list_letter_color in podID_pod_letter_color.items():
          if letter_color in list_letter_color:
            order_podID[order] += [podID]
  return order_podID

#Creating a function to get amount of items for particular order:
def get_order_podID_itemcount(orders = orders_list):
  order_itemID_podID = get_order_itemID_podID()
  order_podID_itemcount = {}
  for order in orders_list:
    order_podID_itemcount[order] = {}
    for itemID, pos in order.Positions.items():
      podID = order_itemID_podID[order][itemID]
      order_podID_itemcount[order][podID] = float(pos.Count)
  return order_podID_itemcount

#create a dictionary podID -> itemID
#This dictionary is for dedicated policy, for mix policy for each podID will be list of itemID
def get_podID_itemID(orders = orders_list):
  order_itemID_item_color_letter = get_order_itemID_letter_color(orders_list)
  podID_pod_letter_color = get_podID_pod_letter_color()
  podID_itemID = {}
  for order in orders:
    for itemID, letter_color in order_itemID_item_color_letter[order].items():
      for podID, list_letter_color in podID_pod_letter_color.items():
        if letter_color in list_letter_color:
          podID_itemID[podID] = itemID
  return podID_itemID

#To get the weights of the orders
def get_order_weights(orders = orders_list):
  order_weights = {}
  for order in orders:
    order_weights[order] = extract_order_weight(order, ItemDescriptions = _demo.warehouseInstance.ItemDescriptions)
  return order_weights

## Assigning orders to the packing stations (depots)

In [199]:
def orders_to_depots_recursive(orders_, depot_to_order_distances, depot_assignment_dict,
                               orders_str, orders_match, orders_match_rev, w_i, w_o, c, B_star, depots):
    #creating list of mu:
    while orders_ != []:
      mu_list = []
      priority_list = []
      for order in orders_:
          d = list(depot_to_order_distances[order].values()) # list of distances from order to depots
          depots_list = list(depot_to_order_distances[order].keys())
          lists_to_sort = zip(d, depots_list)
          sorted_lists_d = sorted(lists_to_sort)  # sorted according to d
          d_sorted = [elem for elem,_ in sorted_lists_d]
          depots_list_sorted = [elem for _,elem in sorted_lists_d]
          priority = depots_list_sorted[0]
          #getting priority for assignment before balancing:
          if len(d) > 1:
              mu = d_sorted[1] - d_sorted[0] #difference in distances from order to depots between shortest and second shortest
              mu_list.append(mu)
              priority_list.append(priority)
          #if there is only one choice of depot then assign order to it:
          elif len(d) == 1:
              depot_assignment_dict[priority][order] = 1  # assigning the order to depot
              w_i[priority] = w_i[priority] + w_o[order] #adding the weight to the depot
              orders_str.remove(orders_match_rev[order])
              orders_.remove(order)
      #sort orders according to mu
      lists_to_sort = zip(mu_list, orders_str, priority_list)
      sorted_lists = sorted(lists_to_sort, reverse=True )#sorted according to mu in discending order
      orders_str_sorted = [elem for _1, elem, _3 in sorted_lists]
      orders_sorted = []
      for order_str in orders_str_sorted:
        orders_sorted.append(orders_match[order_str])
      priority_list_sorted = [elem for _1, _2, elem in sorted_lists]
      for index, order in enumerate(orders_sorted):
          if orders_ == [] or len(list(depot_to_order_distances[order].values())) == 1:
            break
          depot_ = priority_list_sorted[index]
          if (w_i[depot_] + w_o[order]) / c <= B_star / len(depots):
              depot_assignment_dict[depot_][order] = 1 #assigning the order to depot
              w_i[depot_] = w_i[depot_] + w_o[order]
              orders_str.remove(orders_match_rev[order])
              orders_.remove(order)
          else:
              depot_to_order_distances[order].pop(depot_)
            #orders_to_depots_recursive(orders_, depot_to_order_distances, depot_assignment_dict,
            #                                     orders_str, orders_match, orders_match_rev, w_i, w_o, c, B_star, depots)


#Assigning orders to depots 
def assigning_orders_to_depots(depots = _demo.warehouseInstance.OutputStations.keys(), 
                               podID_list = _demo.warehouseInstance.Pods.keys(), 
                               orders_list = orders_list, distances = _demo.distance_ij, c = 18):
  orders_ = orders_list.copy()
  order_podID = get_order_podID(orders_)
  order_weights = get_order_weights(orders_)
  orders_str = ['OC'+ str(i+1) for i in range(len(orders_))] #we need this list to sort orders
  orders_match = dict(zip(orders_str, orders_)) #we need this dictionary to get correspondent order for order_str after sorting
  orders_match_rev = dict(zip(orders_,orders_str)) #reversed dictionary to delete orders from both lists after assignment
  #set of distances from depot to order:
  depot_to_order_distances = {}
  for order in orders_:
    depot_to_order_distances[order] = {}
    for depot in depots:
      tmp = 0
      for podID in order_podID[order]:
        tmp += distances[(depot, podID)]
      depot_to_order_distances[order][depot] = tmp
  w_o = order_weights #weights of the orders
  cum_orders_weight = sum([w_o[order] for order in orders_]) #total weight of the orders
  #min number of batches to get all orders
  B_star = math.ceil(cum_orders_weight / c)
  # Initialization
  depot_assignment_dict = {
      depot: dict(zip(orders_, [0 for i in range(len(orders_))]))
      for depot in depots}  # set all orders not assigned
  weight_stations = [sum([depot_assignment_dict[depot][order] * w_o[order] for order in orders_]) for depot in depots]
  w_i = dict(zip(depots, weight_stations))  # weights of the stations
  ###
  #Recursive function to assign orders to depots
  orders_to_depots_recursive(orders_, depot_to_order_distances, 
                            depot_assignment_dict, orders_str, orders_match, 
                            orders_match_rev, w_i, w_o, c, B_star, depots)
  ###
  #Changing the format to more convenient one
  depot_order_assignmnet = defaultdict(list)
  for depot, order_value in depot_assignment_dict.items():
    for order, value in order_value.items():
      if value == 1:
        depot_order_assignmnet[depot] += [order]
  return depot_order_assignmnet , orders_match_rev


In [200]:
depot_order_assignmnet, orders_match_ID = assigning_orders_to_depots(orders_list = orders_list)
depot_order_assignmnet

defaultdict(list,
            {'OutD0': [<rafs_instance.Order at 0x7f95f0cf3610>,
             'OutD1': [<rafs_instance.Order at 0x7f95f0cf3690>,
              <rafs_instance.Order at 0x7f95f0cecad0>]})

## Assigning orders to batches

In [201]:
#The function to get dictionary of the batch weights
def get_depot_batch_weight():
  batches_of_depots = get_batches_of_depots()
  depot_batch_weight = {}
  order_weights = get_order_weights()
  for depot in batches_of_depots.keys():
    depot_batch_weight[depot] = {}
    for batchnr, batch in batches_of_depots[depot].items():
      weight = 0
      for order in batch:
        weight += order_weights[order]
      depot_batch_weight[depot][batchnr] =  weight
  return depot_batch_weight

In [202]:
#Creating a dictionary with all distances between orders calculated
def distance_matrix(O_i, distances = _demo.distance_ij):
  order_podID = get_order_podID()
  distances_between_orders = {}
  for order_1 in O_i:
    distances_between_orders[order_1] = {}
    for order_2 in O_i:
      distance_orders = 0
      for podID_1 in order_podID[order_1]:
        distance_items = 0
        for podID_2 in order_podID[order_2]:
          distance_items += distances[(podID_1,podID_2)]       
        distance_orders += distance_items
      distances_between_orders[order_1][order_2] = distance_orders
  return distances_between_orders

#The function to find the closest orders from the batch
def new_order_of_batch(O_i, O_temp, distances_between_orders):
  if O_temp == []:
    o_batch_new = random.choice(O_i)
  all_distances_list = []
  # loop through orders unassigned to a batch in this depot
  for candidate_order in O_i:
    distance_candidate = 0
    # loop through the orders already assigned to this batch in this depot
    for order_temp in O_temp:
      distance_candidate += distances_between_orders[candidate_order][order_temp]
    all_distances_list.append(distance_candidate)
  min_distance = min(all_distances_list) 
  min_distance_ind = all_distances_list.index(min_distance) 
  o_batch_new = O_i[min_distance_ind]
  return o_batch_new

#Function to assign orders from one depot to batches:
def assigning_orders_to_batches(depot):
  batchnr = 1
  batches_of_orders = {}
  packing_station_orders, _ = assigning_orders_to_depots()
  O_i = packing_station_orders[depot]
  order_weights = get_order_weights(O_i)
  distances_between_orders = distance_matrix(O_i, distances = _demo.distance_ij)
  c = 18 
  for order in O_i:
    if order_weights[order] > c:#we found out that in 20_5a and 20_5b there are orders heavier than 18
    #print(o_batch_new)
      O_i.remove(order) #make sure we do not allow orders heavier than batch capacity
  while O_i != []: #iterate till all orders of the station are assigned
    weight = 0
    O_temp = []#list of orders in the batch
    i = 0 
    while any(weight + order_weights[o] <= c for o in O_i): #control weight capacity of the batch
    #find minimum distance order for the batch:
      o_batch_new = new_order_of_batch(O_i, O_temp, distances_between_orders)
      if weight + order_weights[o_batch_new] <= c:
        o_batch_new = o_batch_new
      else:
        for order in O_i:#searching for other orders to fit in the batch
          if weight + order_weights[order] <= c:
            O_i_add = O_i.copy()
            O_i_add.remove(o_batch_new)
            o_batch_new = new_order_of_batch(O_i_add, O_temp, distances_between_orders)
      weight += order_weights[o_batch_new]
      O_temp.append(o_batch_new)
      O_i.remove(o_batch_new)        
    batches_of_orders[batchnr] = O_temp
    batchnr += 1
  return batches_of_orders

In [203]:
#getting the assignment of all orders to batches:
def get_batches_of_depots(packing_stations = packing_stations):
  batches_of_depots = {}
  for depot in packing_stations:
    batches_of_depots[depot] = defaultdict(list)
    batches_of_depots[depot] = assigning_orders_to_batches(depot)
  return batches_of_depots

In [204]:
batches_of_depots = get_batches_of_depots()
batches_of_depots

{'OutD0': {1: [<rafs_instance.Order at 0x7f95f0cf3610>,
  2: [<rafs_instance.Order at 0x7f95f0cf1850>,
  3: [<rafs_instance.Order at 0x7f95f0cf0ad0>],
  4: [<rafs_instance.Order at 0x7f95f0cf0d90>],
  5: [<rafs_instance.Order at 0x7f95f0cec9d0>,
   <rafs_instance.Order at 0x7f95f0d5d1d0>]},
 'OutD1': {1: [<rafs_instance.Order at 0x7f95f0cf3690>,
  2: [<rafs_instance.Order at 0x7f95f0cf3590>,
  3: [<rafs_instance.Order at 0x7f95f0cf1dd0>,
  4: [<rafs_instance.Order at 0x7f95f0cf1d10>,
   <rafs_instance.Order at 0x7f95f0cecad0>]}}

In [205]:
get_depot_batch_weight()

{'OutD0': {1: 17.52, 2: 13.55, 3: 17.79, 4: 13.870000000000001, 5: 17.57},
 'OutD1': {1: 15.3, 2: 17.529999999999998, 3: 16.41, 4: 15.19}}

## Searching for the best routes for each batch

In [206]:
# searching for the best route of the batches
def find_best_route(batch, depot):
    # batch is the list of orders
    remaining = []  # the list of the podID for all items in the batch
    order_podID = get_order_podID()
    for order in batch:
        for podID in order_podID[order]:
            if not podID in remaining:
                remaining.append(podID)
    start = depot  # the start and end point is packing station
    head = depot  # the starting point is the depot
    path = [head]  # keep track of the locations visited
    distance = 0.0
    while len(remaining) > 0:  # loop until all locations are visited in the order
        ds = {}
        #print(remaining)
        for position in remaining:
            ds[position] = distances[(head, position)]
        best, best_distance = sorted(ds.items(), key=lambda x: x[1])[0]  # select shortest path
        remaining.remove(best)  # remove the location from the not visited dict.
        head = best  # update the head of the path to the current location of the cobot
        path += [best]  # update the path taken by the cobot
        distance += best_distance  # update the distance traveled by the cobot
    distance += distances[(head, start)]  # add the distance for going back to the packing station
    path += [start]  # adding the packing station to the path
    return path, distance


# find init solution:
def initial_solution(packing_stations = packing_stations):
  batches_of_depots = get_batches_of_depots(packing_stations)
  depot_batch_bestpath = {}
  depot_batch_bestdistance = {}
  for depot in packing_stations:
      depot_batch_bestpath[depot] = {}
      depot_batch_bestdistance[depot] = {}
      for batchnr, batch in batches_of_depots[depot].items():
          bestpath, bestdistance = find_best_route(batch, depot)
          depot_batch_bestpath[depot][batchnr] = bestpath
          depot_batch_bestdistance[depot][batchnr] = bestdistance
  sum_of_batch_distances = [sum(depot_batch_bestdistance[depot].values()) for depot in packing_stations]
  return depot_batch_bestpath, sum_of_batch_distances 

In [207]:
initial_solution()

({'OutD0': {1: ['OutD0',
    '150',
    '129',
    '172',
    '14',
    '36',
    '35',
    '47',
    '51',
    '23',
    '215',
    'OutD0'],
   2: ['OutD0', '150', '128', '74', '79', '14', '35', '215', '217', 'OutD0'],
   3: ['OutD0',
    '150',
    '215',
    '74',
    '67',
    '79',
    '14',
    '35',
    '23',
    '266',
    'OutD0'],
   4: ['OutD0', '150', '128', '172', '217', '215', '67', '51', '36', 'OutD0'],
   5: ['OutD0',
    '150',
    '128',
    '74',
    '67',
    '14',
    '36',
    '35',
    '51',
    '23',
    '215',
    '349',
    'OutD0']},
  'OutD1': {1: ['OutD1',
    '23',
    '51',
    '47',
    '14',
    '74',
    '67',
    '150',
    '217',
    'OutD1'],
   2: ['OutD1',
    '144',
    '172',
    '128',
    '150',
    '35',
    '36',
    '47',
    '51',
    '28',
    '266',
    'OutD1'],
   3: ['OutD1', '266', '349', '129', '150', '67', '14', '35', '47', 'OutD1'],
   4: ['OutD1',
    '172',
    '128',
    '150',
    '35',
    '14',
    '47',
    '51',
    '23',

## Makespan algorithm

### Assigning pods to zones

In [208]:
pods_dict = _demo.warehouseInstance.Pods

In [209]:
#We need a function to make our solution without batchnumbers, as one path:
def flatten_batches(depot_batch_bestpath):
  cobot_path = defaultdict(list)
  for cobot in depot_batch_bestpath.keys():
    for batch in depot_batch_bestpath[cobot].values():
      for podID in batch:
        if cobot_path[cobot] != [] and cobot_path[cobot][-1] == podID:
          continue
        cobot_path[cobot].append(podID)
  return cobot_path 

In [210]:
#Creating neccessary data structures:
#creating a list of zones:
def get_zones(pods_dict = pods_dict):
  if len(list(pods_dict.keys())) == 24:
    zones = ['zone_'+ str(i+1) for i in range(2)]
  else:
    zones = ['zone_'+ str(i+1) for i in range(6)]
  return zones


In [211]:
#dictionary PodID -> zone:
def get_podID_zone(pods_dict = pods_dict):
  zones = get_zones(pods_dict)
  podID_list = list(pods_dict.keys())
  podID_zone = {}
  for ind, zone in enumerate(zones):
    num_zone = ind + 1
    start_element = int((num_zone - 1)*len(podID_list)/len(zones))
    end_element = int(num_zone*len(podID_list)/len(zones))
    for podID in podID_list[start_element:end_element]:
      podID_zone[podID] = zone  
  return podID_zone

In [212]:
  #dictionary that contains instead of items their corresponding zones, represented as one path (without batches):
  def get_cobot_item_zones(solution):
    podID_zone = get_podID_zone()
    cobot_item_zones = defaultdict(list)
    for depot in solution.keys():
      for batchnr, batch in solution[depot].items():
        cobot_item_zones[depot].append(depot)
        for podID in batch[1:-1]:   
          cobot_item_zones[depot].append(podID_zone[podID])
      cobot_item_zones[depot].append(depot)  
    return cobot_item_zones

In [213]:
def get_cobot_zone_sequence(solution):
    podID_zone = get_podID_zone()
    cobot_zone_sequence = defaultdict(list)
    for depot in solution.keys():
      for batchnr, batch in solution[depot].items():
        for podID in batch[1:-1]:  
          cobot_zone_sequence[depot].append(podID_zone[podID]) 
        cobot_zone_sequence[depot].append(depot)
      cobot_zone_sequence[depot] = [i[0] for i in groupby(cobot_zone_sequence[depot])]
    return cobot_zone_sequence


###  Dictionary to collect right amount of items from shelfs on batch

In [214]:
#Creating a dictionary to get amount of item to pick grom each pod:
def get_depot_batch_podID_itemcount(solution_orders, packing_stations = packing_stations, orders = orders_list):
  order_podID_itemcount = get_order_podID_itemcount(orders)
  depot_batch_podID_itemcount = {}  
  for depot in batches_of_depots.keys():
    depot_batch_podID_itemcount[depot] = {}
    for batchnr, orders_of_batch in solution_orders[depot].items():
      depot_batch_podID_itemcount[depot][batchnr] = {}
      for order in orders_of_batch:
        for podID, itemcount in order_podID_itemcount[order].items():
          if podID in depot_batch_podID_itemcount[depot][batchnr].keys():
            depot_batch_podID_itemcount[depot][batchnr][podID] += itemcount
          else:
            depot_batch_podID_itemcount[depot][batchnr][podID] = itemcount
  return depot_batch_podID_itemcount


#For our purposes we need flat version of the dictionary above:
def get_depot_itemcount(solution, solution_orders):
  depot_batch_podID_itemcount = get_depot_batch_podID_itemcount(solution_orders)
  depot_itemcount = defaultdict(list)
  for depot in depot_batch_podID_itemcount.keys():
    for batchnr in depot_batch_podID_itemcount[depot].keys():
      for podID in solution[depot][batchnr]:
        if podID == depot:
          continue
        else: depot_itemcount[depot].append(depot_batch_podID_itemcount[depot][batchnr][podID])
  return depot_itemcount

### Algorithm

In [215]:
#we need function to calculate serving time of a cobot in a zone:
def picker_serving(zone, cobot, cobot_paths, cobot_item_zones, picker_speed, picking_time, distances, depot_itemcount):
  t_picking = picking_time*depot_itemcount[cobot][0]
  t_serving = t_picking
  del depot_itemcount[cobot][0]
  while cobot_item_zones[cobot][1] == zone:
    t_picker_travel = distances[(cobot_paths[cobot][0],cobot_paths[cobot][1])]/picker_speed
    t_picking = picking_time*depot_itemcount[cobot][0]
    t_serving = t_serving + t_picker_travel + t_picking
    del cobot_item_zones[cobot][0]
    del depot_itemcount[cobot][0]
    del cobot_paths[cobot][0]
  last_pod = cobot_paths[cobot][0]
  return last_pod, t_serving

In [216]:
#function that to calculate the time of cobot spent in the zone on serving
def zone_time(t_start, zone, cobot, cobot_paths, cobot_item_zones, cobot_zone_sequence, picker_speed, picking_time, distances, depot_itemcount, zones_occupation_time):
  #deleting previous waypoints:
  del cobot_item_zones[cobot][0]
  del cobot_paths[cobot][0]
  #calculating serving time (picking time for all items of the zone):
  last_pod, t_serving = picker_serving(zone, cobot, cobot_paths, cobot_item_zones, 
                                       picker_speed, picking_time, distances, depot_itemcount)
  #calculating finishing time for the zone
  t_finish = t_start + t_serving
  zones_occupation_time[zone] = (t_start, t_finish)
  #updating times and position of the cobot
  del cobot_zone_sequence[cobot][0]
  return t_finish, last_pod

In [217]:
#the function for calculating the makespan 
def get_makespan(solution, solution_orders):
  solution_ = solution.copy()
  zones = get_zones(pods_dict)
  ###
  zones_occupation_time = {}
  t_start = 0
  t_finish = 0
  for zone in zones:
    zones_occupation_time[zone] = (t_start, t_finish)
  zones_occupation_time
  ###
  picker_position = {}
  for zone in zones:
    picker_position[zone] = 0
  ###
  depot_occupation_time = {}
  t_pack_start = 0
  t_pack_finish = 0
  for depot in packing_stations:
    depot_occupation_time[depot] = (t_pack_start, t_pack_finish)
  #parameters:
  cobot_speed = 2
  picker_speed = 1.3
  picking_time = 3 
  t_prep = 30 
  t_unload = 20 
  t_packing = 60 
  cobots = packing_stations.copy()#to be able to delete cobots that finished their jobs
  ###
  #neccessary data structures:
  podID_zone = get_podID_zone()#to extract zone of the item
  batches_of_depots = solution_orders
  depot_batch_bestpath = solution_
  depot_itemcount = get_depot_itemcount(depot_batch_bestpath, batches_of_depots)#to know how many items to pick
  cobot_paths = flatten_batches(depot_batch_bestpath)#cobot path
  cobot_zone_sequence = get_cobot_zone_sequence(depot_batch_bestpath)#to know next zone to visit
  cobot_item_zones = get_cobot_item_zones(depot_batch_bestpath)#path with zones instead of corresponding items
  ###
  waiting_time = defaultdict(list)#we will append here all waiting times before each new zone for cobot
  #creating a dictionary to set current time to time that was spent on preparation
  t_current = {}
  for cobot in packing_stations:
    t_current[cobot] = t_prep
  #The body of the algorithm:
  while any(cobot_zone_sequence[cobot]  != [] for cobot in packing_stations):#till all zones of the path are visited (depots are also zones)
    #determining the next zone for each cobot:
    next_zone = []
    for cobot in packing_stations:
      if cobot_zone_sequence[cobot] == []:
        if cobot in cobots:
          cobots.remove(cobot)
      else:
        next_zone.append(cobot_zone_sequence[cobot][0])
    if len(next_zone) != len(set(next_zone)):#check if cobots plan to go to the same zone
      next_zone_ = next_zone[0]
      t_arrival_next_zone = []
      for cobot in packing_stations:
        t_arrival_next_zone.append(t_current[cobot] + distances[(cobot_paths[cobot][0], cobot_paths[cobot][1])]/cobot_speed)
      min_time = min(t_arrival_next_zone) 
      min_time_ind = t_arrival_next_zone.index(min_time) 
      cobot_first = packing_stations[min_time_ind]
      cobot_second = packing_stations[min_time_ind-1]
      t_travel = distances[(cobot_paths[cobot_first][0], cobot_paths[cobot_first][1])]/cobot_speed
      #in case if picker needs more time to get to the first item than first cobot
      if picker_position[next_zone_] != 0:#check if it is the first item for cobot
        t_travel_picker = distances[(picker_position[next_zone_], cobot_paths[cobot_first][1])]/picker_speed
        if t_current[cobot_first] + t_travel  < zones_occupation_time[next_zone_][1] + t_travel_picker: #if picker will arrive later than cobot can
          t_start = zones_occupation_time[next_zone_][1] + t_travel_picker
          waiting_time[cobot_first].append(zones_occupation_time[next_zone_][1] + t_travel_picker - t_current[cobot_first] - t_travel)
        else:
          t_start = t_current[cobot_first] + t_travel
      else:  
        t_start = t_current[cobot_first] + t_travel  
      t_finish, picker_position[next_zone_] = zone_time(t_start, next_zone_, cobot_first, cobot_paths, cobot_item_zones,
                                         cobot_zone_sequence, picker_speed, picking_time, distances, depot_itemcount, zones_occupation_time)
      t_current[cobot_first] = t_finish
      ###Calculating waiting time of the second cobot:
      t_travel = distances[(cobot_paths[cobot_second][0], cobot_paths[cobot_second][1])]/cobot_speed
      t_travel_picker = distances[(picker_position[next_zone_], cobot_paths[cobot_second][1])]/picker_speed
      if t_current[cobot_second] + t_travel  <= t_finish + t_travel_picker: 
        waiting_time[cobot_second].append(t_finish + t_travel_picker - t_travel - t_current[cobot_second])
        t_current[cobot_second] =  t_current[cobot_second] + waiting_time[cobot_second][-1]
      t_start = t_current[cobot_second] + t_travel
      t_current[cobot_second], picker_position[next_zone_] = zone_time(t_start, next_zone_, cobot_second, cobot_paths, 
                                         cobot_item_zones, cobot_zone_sequence, picker_speed, picking_time, distances, depot_itemcount, zones_occupation_time)
    else:
      for ind, cobot in enumerate(cobots):
          t_travel = distances[(cobot_paths[cobot][0], cobot_paths[cobot][1])]/cobot_speed
          if next_zone[ind] == cobot: #check if the next zone is depot
            if t_current[cobot] + t_travel >= depot_occupation_time[cobot][1]: #if cobot comes when packer finished packing
              del cobot_item_zones[cobot][0]
              del cobot_paths[cobot][0]
              if len(cobot_zone_sequence[cobot]) == 1: #if it is the end of the last tour of the cobot
                t_pack_start = t_current[cobot] + t_travel + t_unload #we do not have preparation
                t_current[cobot] = t_current[cobot] + t_travel + t_unload 
              else:
                t_pack_start = t_current[cobot] + t_travel + t_prep + t_unload
                t_current[cobot] = t_current[cobot] + t_travel + t_prep + t_prep + t_unload
              t_pack_finish = t_pack_start + t_packing
              depot_occupation_time[cobot] = (t_pack_start, t_pack_finish)
              del cobot_zone_sequence[cobot][0]
            else: #if packer still packing, cobot needs to wait
              t_pack_finish = depot_occupation_time[cobot][1]
              del cobot_item_zones[cobot][0]
              del cobot_paths[cobot][0]
              waiting_time[cobot].append(t_pack_finish - t_travel - t_current[cobot]) 
              t_current[cobot] += waiting_time[cobot][-1]
              if len(cobot_zone_sequence[cobot]) == 1:#if it is the end of the last tour of the cobot
                t_pack_start = t_current[cobot] + t_travel + t_unload
                t_current[cobot] = t_current[cobot]   + t_travel + t_unload
              else:
                t_pack_start = t_current[cobot] + t_travel + t_prep + t_unload
                t_current[cobot] = t_current[cobot] + t_travel + t_unload + t_prep
              t_pack_finish = t_pack_start + t_packing
              depot_occupation_time[cobot] = (t_pack_start, t_pack_finish)
              del cobot_zone_sequence[cobot][0]
          else:
            if picker_position[next_zone[ind]] != 0:
              t_travel_picker = distances[(picker_position[next_zone[ind]], cobot_paths[cobot][1])]/picker_speed
              if t_current[cobot] + t_travel  <= zones_occupation_time[next_zone[ind]][1] + t_travel_picker: #if picker will arrive later than cobot can
                t_start = zones_occupation_time[next_zone[ind]][1] + t_travel_picker
                waiting_time[cobot].append(zones_occupation_time[next_zone[ind]][1] + t_travel_picker - t_current[cobot] - t_travel)
              else:
                t_start = t_current[cobot] + t_travel
            else:  
              t_start = t_current[cobot] + t_travel
            t_current[cobot], picker_position[next_zone[ind]] = zone_time(t_start, next_zone[ind], cobot, cobot_paths, 
                                         cobot_item_zones, cobot_zone_sequence, picker_speed, picking_time, distances, depot_itemcount, zones_occupation_time)
  total_waiting_time = sum([sum(waiting_time[depot]) for depot in packing_stations])
  return t_pack_finish, total_waiting_time

## Greedy Results:

In [218]:
init_solution,_ = initial_solution()
init_solution_orders = get_batches_of_depots()
init_solution_makespan, init_solution_waiting_time= get_makespan(init_solution, init_solution_orders )
print('+------------------------ Greedy RESULTS -------------------------+\n')
print(f'  initial solution: {init_solution}\n')
print(f'  initial solution waiting time : {init_solution_waiting_time}\n')
print(f'  initial solution makespan: {init_solution_makespan}\n')

print('+-------------------------- END ---------------------------+')

+------------------------ Greedy RESULTS -------------------------+

  initial solution: {'OutD0': {1: ['OutD0', '150', '129', '172', '14', '36', '35', '47', '51', '23', '215', 'OutD0'], 2: ['OutD0', '150', '128', '74', '79', '14', '35', '215', '217', 'OutD0'], 3: ['OutD0', '150', '215', '74', '67', '79', '14', '35', '23', '266', 'OutD0'], 4: ['OutD0', '150', '128', '172', '217', '215', '67', '51', '36', 'OutD0'], 5: ['OutD0', '150', '128', '74', '67', '14', '36', '35', '51', '23', '215', '349', 'OutD0']}, 'OutD1': {1: ['OutD1', '23', '51', '47', '14', '74', '67', '150', '217', 'OutD1'], 2: ['OutD1', '144', '172', '128', '150', '35', '36', '47', '51', '28', '266', 'OutD1'], 3: ['OutD1', '266', '349', '129', '150', '67', '14', '35', '47', 'OutD1'], 4: ['OutD1', '172', '128', '150', '35', '14', '47', '51', '23', '28', '266', 'OutD1']}}

  initial solution waiting time : 93.06923076923063

  initial solution makespan: 1115.761538461538

+-------------------------- END --------------------

In [219]:
# to swap positions of two items in a batch:
def swapPositions(list, pos1, pos2):
  list_copy = list.copy()
  list_copy[pos1], list_copy[pos2] = list_copy[pos2], list_copy[pos1]
  return list_copy

In [220]:
# trying to find a neighbor for shuffling the orders between batches and items inside the orders:
def find_neigbour_solution_batches(solution):
  # initialize all the needed data:
  order_weights = get_order_weights()
  batches_of_depots = get_batches_of_depots(packing_stations)
  batches_in_solution = batches_of_depots.copy()
  solution_0 = solution.copy()
  neighbor_solution = {}
  solution_order_list_station1 = []
  solution_order_list = []
  depot_batch_bestpath = {}
  depot_batch_bestdistance = {}
  # get a list of the orders for each station:
  for station, batch_number_orders in batches_in_solution.items():
    if station == 'OutD0':
      for num, orders_list in batch_number_orders.items():
        if len(orders_list) > 1:
          for i in range(len(orders_list)):
           solution_order_list.append(orders_list[i])
        else:
          solution_order_list.append(orders_list[0])
      # start the update of the batch assignment(here we do it randomly)
      batchnr = 1
      new_batches = dict([])
      O_i = solution_order_list.copy()
      random.shuffle(O_i) # making sure we change the order of the orders in the list
      c = 18
      while O_i != []: #iterate till all orders of the station are assigned
        weight = 0
        O_temp = []#list of orders in the batch
        while any(weight + order_weights[o] <= c for o in O_i): #control weight capacity of the batch
          #pdb.set_trace()
          o_batch_new_assign = None
          pos = random.randint(0, len(O_i)-1)
          o_batch_new = O_i[pos]
          if order_weights[o_batch_new] > c:#we found out that in 20_5a and 20_5b there are orders heavier than 18
            O_i.remove(o_batch_new) #make sure we do not allow orders heavier than batch capacity
          elif weight + order_weights[o_batch_new] <= c:
            o_batch_new_assign = o_batch_new
          else:
            for order in O_i:
              if weight + order_weights[order] <= c:
                o_batch_new_assign = order
          if o_batch_new_assign == None:
              break
          weight += order_weights[o_batch_new_assign]
          O_temp.append(o_batch_new_assign)
          O_i.remove(o_batch_new_assign)    

        new_batches[batchnr] = O_temp
        batchnr += 1
      neighbor_solution[station] = new_batches
    else: 
      # same as above but for the other station:
      for num, orders_list in batch_number_orders.items():
        if len(orders_list) > 1:
          for i in range(len(orders_list)):
            solution_order_list_station1.append(orders_list[i])
        else:
          solution_order_list_station1.append(orders_list[0])
      batchnr = 1
      new_batches = dict([])
      O_i = solution_order_list_station1.copy()
      random.shuffle(O_i)
      c = 18
      while O_i != []: #iterate till all orders of the station are assigned
        weight = 0
        O_temp = []#list of orders in the batch
        while any(weight + order_weights[o] <= c for o in O_i ) and O_i != []: #control weight capacity of the batch
          #pdb.set_trace()
          o_batch_new_assign = None
          pos = random.randint(0, len(O_i)-1)
          o_batch_new = O_i[pos]
          if order_weights[o_batch_new] > c:#we found out that in 20_5a and 20_5b there are orders heavier than 18
            O_i.remove(o_batch_new) #make sure we do not allow orders heavier than batch capacity
          elif weight + order_weights[o_batch_new] <= c:
            o_batch_new_assign = o_batch_new
          else:
            for order in O_i:
              if weight + order_weights[order] <= c:
                o_batch_new_assign = order
          if o_batch_new_assign == None:
              break
          weight += order_weights[o_batch_new_assign]
          O_temp.append(o_batch_new_assign)
          O_i.remove(o_batch_new_assign)    

        new_batches[batchnr] = O_temp
        batchnr += 1
      neighbor_solution[station] = new_batches
    # now we have assigned orders to batches, saved in neighbor_solution dictionary

    # we will now determin the bath in greedy way for each batch
    # but we will also switch two items positions to add some randomness
    depot_batch_bestpath[station] = {}
    depot_batch_bestdistance[station] = {}
    for batchnr, batch in neighbor_solution[station].items():
      bestpath, bestdistance = find_best_route(batch, station)
      depot_batch_bestpath[station][batchnr] = bestpath
      pos1 = random.randint(1,len(bestpath) - 2)
      pos2 = random.randint(1, len(bestpath) - 2)
      depot_batch_bestpath[station][batchnr] = swapPositions(bestpath,pos1,pos2)
  return depot_batch_bestpath, neighbor_solution

In [221]:
# finding a local neighbour solution by randomly exchanging positions of items inside batches:
def find_neigbour_solution_local(solution):
    solution_ = solution.copy()
    depot_batch_bestpath = {}
    depot_batch_bestdistance = {}
    for station in solution_.keys():
      depot_batch_bestpath[station] = {}
      depot_batch_bestdistance[station] = {}
      for batchnr, batch in solution_[station].items():
        pos1 = random.randint(1,len(batch) - 2)
        pos2 = random.randint(1, len(batch) - 2)
        depot_batch_bestpath[station][batchnr] = swapPositions(batch,pos1,pos2)
    return depot_batch_bestpath

# itterated local search:

In [222]:
 # Local Search:
def local_search_solution(iterations = 100):
  # initial solution:
  solution, _ = initial_solution()
  solution_orders = get_batches_of_depots()
  solution_makespan, solution_waiting_time = get_makespan(solution, solution_orders)
  for i in range(iterations):
    # local neighbour solution:
    solution_temp = find_neigbour_solution_local(solution)
    solution_temp_orders = get_batches_of_depots()
    neighbor_makespan_temp, neighbor_waiting_time = get_makespan(solution_temp, solution_temp_orders)
    # check if we got a better local neighbour:
    if solution_makespan > neighbor_makespan_temp :
      solution = solution_temp
      solution_makespan = neighbor_makespan_temp
      solution_orders = solution_temp_orders
      solution_waiting_time = neighbor_waiting_time
  return solution , solution_orders ,solution_makespan, solution_waiting_time

def iterated_local_search_solution(solution , solution_orders ,solution_makespan,solution_waiting_time, iterations = 100):
  solution = solution.copy()
  solution_orders = solution_orders.copy()
  solution_waiting_time = solution_waiting_time
  for i in range(iterations):
    # start the perturbation:
    solution_temp, solution_temp_orders = find_neigbour_solution_batches(solution)
    neighbor_makespan_temp, neighbor_waiting_time = get_makespan(solution_temp, solution_temp_orders)
    # take the better solution with less makespan
    if solution_makespan > neighbor_makespan_temp :
      solution = solution_temp
      solution_orders = solution_temp_orders
      solution_makespan = neighbor_makespan_temp
      solution_waiting_time = neighbor_waiting_time
  return solution, solution_orders, solution_makespan, neighbor_waiting_time

local_solution , local_solution_orders ,local_solution_makespan, local_solution_waiting_time= local_search_solution(iterations = 10000)
ils_solution, ils_solution_orders ,ils_solution_makespan, ils_solution_waiting_time = iterated_local_search_solution(init_solution, init_solution_orders  ,init_solution_makespan,init_solution_waiting_time ,iterations = 10000)

print('+------------------------ Local Search RESULTS -------------------------+\n')

print(f'local solution: {local_solution}\n', f'iterated solution : {ils_solution}')
print(f'local solution makespan: {local_solution_makespan}\n',f'iterated solution makespan : {ils_solution_makespan}')
print(f'local solution waiting time: {local_solution_waiting_time}\n', f'iterated solution waiting time : {ils_solution_waiting_time}')

print('+-------------------------- END ---------------------------+')



+------------------------ Local Search RESULTS -------------------------+

local solution: {'OutD0': {1: ['OutD0', '150', '129', '14', '35', '36', '172', '47', '23', '51', '215', 'OutD0'], 2: ['OutD0', '128', '150', '74', '79', '14', '35', '217', '215', 'OutD0'], 3: ['OutD0', '150', '266', '23', '67', '74', '14', '35', '79', '215', 'OutD0'], 4: ['OutD0', '150', '128', '67', '51', '215', '217', '172', '36', 'OutD0'], 5: ['OutD0', '150', '128', '74', '67', '14', '36', '35', '23', '51', '215', '349', 'OutD0']}, 'OutD1': {1: ['OutD1', '23', '47', '150', '51', '74', '67', '14', '217', 'OutD1'], 2: ['OutD1', '144', '172', '128', '35', '36', '150', '47', '51', '28', '266', 'OutD1'], 3: ['OutD1', '349', '266', '47', '14', '67', '150', '35', '129', 'OutD1'], 4: ['OutD1', '172', '35', '150', '128', '14', '23', '51', '47', '28', '266', 'OutD1']}}
 iterated solution : {'OutD0': {1: ['OutD0', '74', '128', '129', '150', '79', '14', '35', '51', '217', '215', 'OutD0'], 2: ['OutD0', '150', '128', '172'

# simulated Annealing:

## Defining Simualted Annealing

In [170]:
class minimize():
    def __init__(self, func, solution, solution_orders , find_neigbour_solution, cooling_schedule='linear', step_max=1000, t_min=0, t_max=100, bounds=[], alpha= 0.9, damping=1):

        assert cooling_schedule in ['linear','exponential','logarithmic', 'quadratic'], 'cooling_schedule must be either "linear", "exponential", "logarithmic", or "quadratic"'

        # neighborhood search scheme:
        self.find_neigbour_solution = find_neigbour_solution

        # initialize starting conditions
        self.t = t_max
        self.t_max = t_max
        self.t_min = t_min
        self.step_max = step_max
        self.hist = []
        self.cooling_schedule = cooling_schedule

        self.cost_func = func
        self.solution = solution
        self.solution_orders = solution_orders
        self.bounds = bounds[:]
        self.damping = damping
        self.current_state = self.solution
        self.current_state_orders = self.solution_orders
        self.current_energy, self.current_state_waiting_time = func(self.solution, self.solution_orders)
        self.best_state = self.current_state
        self.best_energy = self.current_energy
        self.best_state_orders = self.current_state_orders
        self.best_waiting_time = self.current_state_waiting_time


        # initialize cooling schedule
        if self.cooling_schedule == 'linear':
            if alpha != None:
                self.update_t = self.cooling_linear_m
                self.cooling_schedule = 'linear multiplicative cooling'
                self.alpha = alpha

            if alpha == None:
                self.update_t = self.cooling_linear_a
                self.cooling_schedule = 'linear additive cooling'

        if self.cooling_schedule == 'quadratic':
            if alpha != None:
                self.update_t = self.cooling_quadratic_m
                self.cooling_schedule = 'quadratic multiplicative cooling'
                self.alpha = alpha

            if alpha == None:
                self.update_t = self.cooling_quadratic_a
                self.cooling_schedule = 'quadratic additive cooling'

        if self.cooling_schedule == 'exponential':
            if alpha == None: self.alpha =  0.8
            else: self.alpha = alpha
            self.update_t = self.cooling_exponential

        if self.cooling_schedule == 'logarithmic':
            if alpha == None: self.alpha =  0.8
            else: self.alpha = alpha
            self.update_t = self.cooling_logarithmic


        # begin optimizing
        self.step, self.accept = 1, 0
        while self.step < self.step_max and self.t >= self.t_min and self.t > 0:
            # get neighbor
            proposed_neighbor , proposed_orders = self.find_neigbour_solution(self.current_state)

            # check energy level of neighbor
            E_n, w_t  = self.cost_func(proposed_neighbor, proposed_orders)
            dE = (E_n - self.current_energy)
                       
            # pdb.set_trace()

            # determine if we should accept the current neighbor
            if random.random() < self.safe_exp(-dE / self.t):
                self.current_energy = E_n
                self.current_state = proposed_neighbor
                self.current_state_orders = proposed_orders
                self.current_state_waiting_time = w_t

                self.accept += 1

            # check if the current neighbor is best solution so far
            #pdb.set_trace()
            if E_n < self.best_energy:
                self.best_energy = E_n
                self.best_state = proposed_neighbor
                self.best_state_orders = proposed_orders
                self.best_waiting_time = w_t

            # persist some info for later
            self.hist.append([
                self.step,
                self.t,
                self.current_energy,
                self.best_energy])

            # update some stuff
            self.t = self.update_t(self.step)
            self.step += 1

        # generate some final stats
        self.acceptance_rate = self.accept / self.step

    def results(self):
        print('+------------------------ RESULTS -------------------------+\n')
        print(f'cooling sched.: {self.cooling_schedule}')
        if self.damping != 1: print(f'       damping: {self.damping}\n')
        else: print('\n')

        print(f'  initial temp: {self.t_max}')
        print(f'    final temp: {self.t:0.6f}')
        print(f'     max steps: {self.step_max}')
        print(f'    final step: {self.step}\n')
        print(f'    final solution: {self.best_state}\n')

        print(f'  final energy: {self.best_energy:0.6f}\n')
        print(f'  final waiting time: {self.best_waiting_time:0.6f}\n')
        print('+-------------------------- END ---------------------------+')

    def get_results(self):
      return self.best_state, self.best_state_orders, self.best_energy, self.best_waiting_time

    # linear multiplicative cooling
    def cooling_linear_m(self, step):
        return self.t_max /  (1 + self.alpha * step)

    # linear additive cooling
    def cooling_linear_a(self, step):
        return self.t_min + (self.t_max - self.t_min) * ((self.step_max - step)/self.step_max)

    # quadratic multiplicative cooling
    def cooling_quadratic_m(self, step):
        return self.t_min / (1 + self.alpha * step**2)

    # quadratic additive cooling
    def cooling_quadratic_a(self, step):
        return self.t_min + (self.t_max - self.t_min) * ((self.step_max - step)/self.step_max)**2

    # exponential multiplicative cooling
    def cooling_exponential(self, step):
        return self.t_max * self.alpha**step

    # logarithmical multiplicative cooling
    def cooling_logarithmic(self, step):
        return self.t_max / (self.alpha * log(step + 1))

    def safe_exp(self, x):
        try: return exp(x)
        except: return 0

## getting results:

In [ ]:
# initial solution:
solution_0,_ = initial_solution()
solution_0_orders = get_batches_of_depots()
# start simulated annealing with specific parameters:
sa_minimizer = minimize(get_makespan,
                        solution_0,solution_0_orders, cooling_schedule='linear' ,find_neigbour_solution = find_neigbour_solution_batches,
                        step_max=10000, t_min=0, t_max=1000, bounds=[], alpha = 0.8 , damping=1)
sa_minimizer.results()

In [173]:
  # extracting results from the simulated annealing:
sa_solution, sa_solution_orders, sa_solution_makespan, sa_solution_waiting_time = sa_minimizer.get_results()

# writing to XML:


## functions needed for XML file:

In [ ]:
def get_order_itemID_podID_letter_color():
  order_itemID_podID = get_order_itemID_podID()
  order_itemID_letter_color = get_order_itemID_letter_color()
  order_itemID_podID_letter_color = {}
  for order in order_itemID_podID.keys():
    order_itemID_podID_letter_color[order] = {}
    for itemID, podID in order_itemID_podID[order].items():
      order_itemID_podID_letter_color[order][itemID] = {}
      #pdb.set_trace()
      order_itemID_podID_letter_color[order][itemID][podID] = order_itemID_letter_color[order][itemID] 
  return order_itemID_podID_letter_color

In [ ]:
def get_depot_batch_distance(solution):
  batches_of_depots = solution
  depot_batch_distance = {}
  for depot in batches_of_depots.keys():
    depot_batch_distance[depot] = {}
    for batchnr, batch in batches_of_depots[depot].items():
      distance = 0
      for i in range(len(batch) - 1):
        distance += distances[(batch[i], batch[i+1] )]
      depot_batch_distance[depot][batchnr] =  distance
  return depot_batch_distance

In [ ]:
def get_depot_batch_weight_solution(solution_orders):
  batches_of_depots = solution_orders
  depot_batch_weight = {}
  order_weights = get_order_weights()
  for depot in batches_of_depots.keys():
    depot_batch_weight[depot] = {}
    for batchnr, batch in batches_of_depots[depot].items():
      weight = 0
      for order in batch:
        weight += order_weights[order]
      depot_batch_weight[depot][batchnr] =  weight
  return depot_batch_weight

In [ ]:
# writing the solution to .xml
def write_solution_to_xml(solution , solution_orders,solution_makespan,solution_waiting_time, filename):
  
    # base element
    root = ET.Element("root")
    # first section "split" contains information about which orders are in which batches, and which batches are assigned to which station (=bot)
    collecting = ET.SubElement(root, "Collecting")
    split = ET.SubElement(collecting, 'Split')
    #packingStationNames = result.keys()
    # write each station as a sub-node of split
    for depot in packing_stations:
        Bot_ID = ET.SubElement(split, "Bot")
        Bot_ID.set("ID", depot)
        # filter the solution so it only contains batches for the right station
        stationSolution = solution[depot]
        # write each batch as a sub-node of Bot_ID
        for batchID, batch in stationSolution.items():
            Batch_ID = ET.SubElement(Bot_ID, "Batch")
            Batch_ID.set("ID", str(batchID))
            # write Orders as the sub-node of Batch_ID
            Orders = ET.SubElement(Batch_ID, "Orders")
            # write each order as sub-node of Batch_ID
            for order in solution_orders[depot][batchID]:
                order_str = orders_match_ID[order]
                Order = ET.SubElement(Orders, "Order")
                Order.text = order_str
                
    # first section "bots" contains detailed information about each bot (station)
    bots = ET.SubElement(collecting, "Bots")
    # write each station as a sub-node of bots
    for depot in packing_stations:
        Bot_ID = ET.SubElement(bots, "Bot")
        Bot_ID.set("ID", depot)
        stationSolution = solution[depot]
        # batches are written in sub-node Batches of Bot_ID
        Batches = ET.SubElement(Bot_ID, "Batches")
        # write each batch as a sub-node of Bot_ID

        for batchID, batch in stationSolution.items():
            Batch_ID = ET.SubElement(Batches, "Batch")
            Batch_ID.set("BatchNumber", str(batchID))

            ### need the distance-----------------------
            Batch_ID.set("Distance", str(solution_distances[depot][batchID]))
            ###------------------------------------------

            ### need the weight:-------------------------
            Batch_ID.set("Weight", str(batches_weights[depot][batchID]))
            ### need the weight--------------------------

            # for each batch, write two sub-nodes: itemsData, edges
            # first write ItemsData
            ItemsData = ET.SubElement(Batch_ID, "ItemsData")
            # ItemsData has a sub-node called Orders
            Orders = ET.SubElement(ItemsData, "Orders")
            # write each order as sub-node of Orders:
            for order in solution_orders[depot][batchID]:
                Order = ET.SubElement(Orders, "Order")
                order_str = orders_match_ID[order]
                Order.set("ID", order_str)
                # write each item in the order as sub-node of Order
                items = items_order_info[order]
                for item_ID, item_info in items.items():
                    Item = ET.SubElement(Order, "Item")
                    # for each item, conclude information about the itemID and the description
                    Item.set("ID", str(item_ID))
                    for pod, c_l in item_info.items():
                      Item.set("Type", str(c_l))
                      Item.set("Pod", str(pod))
         
            # write Edges as sub-node of Batch_ID
            Edges = ET.SubElement(Batch_ID, "Edges")    
            # write every edge of the batch
            edgeIndex = list(range(0, len(solution[depot][batchID])-1))
            for edge in edgeIndex:
                Edge = ET.SubElement(Edges, "Edge")
                Edge.set("StartNode", str(solution[depot][batchID][edge]))
                Edge.set("EndNode", str(solution[depot][batchID][edge + 1]))
    waiting_time = ET.SubElement(collecting, "WaitingTime")
    waiting_time.text = str(solution_waiting_time)
    makespan = ET.SubElement(collecting, "Makespan")
    makespan.text = str(solution_makespan)
    tree = ET.ElementTree(root)
    tree.write(filename)


In [ ]:
# initialising data:
orders = orders_list
pods = pods_dict
order_weights = get_order_weights()
items_order_info = get_order_itemID_podID_letter_color()


for method in ['greedy','iterated_local_search','simulated_annealing']:
  if method == 'greedy':
    solution = init_solution
    solution_orders = init_solution_orders
    solution_makespan = init_solution_makespan
    solution_waiting_time = init_solution_waiting_time
  elif method == 'iterated_local_search':
    solution = ils_solution
    solution_orders = ils_solution_orders
    solution_makespan = ils_solution_makespan
    solution_waiting_time = ils_solution_waiting_time
  elif method == 'simulated_annealing':
    solution = sa_solution
    solution_orders = sa_solution_orders
    solution_makespan = sa_solution_makespan
    solution_waiting_time = sa_solution_waiting_time
  ## get the weights for each batch:
  batches_weights = get_depot_batch_weight_solution(solution_orders)
  ## get the distances for each batch:
  solution_distances = get_depot_batch_distance(solution)
  ## specify the output location:
  outputName = '/content/drive/MyDrive/AN/results/'+ method + '/orders_' + str(orderAmount) + '_mean_' + str(meanItemInOrder) + '_sku_' + str(itemAmount) + str(orderVersion) + '_solution_dedicated_'+ method + '.xml'
  write_solution_to_xml(solution, solution_orders, solution_makespan, solution_waiting_time, outputName)